****Unsupervised ML Example: Cyclohexane****

This notebook serves as an example of how to analyze a simulation trajectory using unsupervised techniques. Here, specifically, we'll be analyzing a simulation of cyclohexane conformations, simulated using quantum-espresso, with metadynamics from PLUMED. The dataset and provenance can be found at:

TBD

Before running this notebook, you will need to install:
    
- [ase](https://wiki.fysik.dtu.dk/ase/index.html)
- [scikit-learn](https://scikit-learn.org/)
- [scikit-cosmo](https://github.com/cosmo-epfl/scikit-cosmo)
- [librascal](https://github.com/cosmo-epfl/librascal)
- [openTSNE](https://opentsne.readthedocs.io/en/latest/)

in addition to standard packages [numpy](https://numpy.org/), [tqdm](https://github.com/tqdm/tqdm), and [matplotlib](https://matplotlib.org/).

In [ ]:
import os
import sys
from functools import partial

import ase
import numpy as np
from ase.io import read, write
from matplotlib import pyplot as plt
from openTSNE import TSNE
from rascal.representations import SphericalInvariants as SOAP
from skcosmo.preprocessing import StandardFlexibleScaler
from sklearn.metrics.pairwise import pairwise_kernels
from sklearn.preprocessing import normalize
from tqdm.auto import tqdm

from utils import set_mpl_fonts, set_cmap

set_mpl_fonts()
cmap = set_cmap()

## Preparing the Data

### Read Data


In [ ]:
# read in the frames from each MD simulation
traj = []
names = ['chair', 'twist-boat', 'boat', 'half-chair', 'planar']
ranges = np.zeros((len(names), 2), dtype=int)
conf_idx = np.zeros(len(names), dtype=int)

for i, n in enumerate(names):
    frames = read(f'./cyclohexane_data/sep_MD/{n}.xyz', ':')

    for frame in frames:
        # wrap each frame in its box
        frame.wrap(eps=1E-10)

        # mask each frame so that descriptors are only centered on carbon (#6) atoms
        mask = np.zeros(len(frame))
        mask[np.where(frame.numbers == 6)[0]] = 1
        frame.arrays['center_atoms_mask'] = mask

    ranges[i] = (len(traj), len(traj) + len(frames))
    conf_idx[i] = len(traj)
    traj = [*traj, *frames]

In [ ]:
# energies of the simulation frames
energy = np.array([a.info['energy_eV'] for a in traj])

# energies of the known conformers
c_energy = np.array([traj[c].info['energy_eV'] for c in conf_idx])

In [ ]:
# extrema for the energies
max_e = max(energy)
min_e = min(energy)

Here we can confirm what our analysis will tell us: the simulation starts in the planar conformation, transitions to the metastable half-chair configuration, then moves through the boat configuration until it ultimately reaches the chair conformation.

In [ ]:
fig, ax = plt.subplots(1, figsize=(3*4.8528, 3*1.2219))

# horizontal lines for each conformation
for n, c, r in zip(names, c_energy, ranges):
    
    ax.plot(range(0, r[1] - r[0]),
            energy[r[0]:r[1]] - min_e,
#             c='lightgrey',
            label=n,
#             linestyle='--',
#             alpha=0.8,
            zorder=-1)
    
ax.legend()
ax.set_xlabel("Simulation Timestep")
ax.set_ylabel("Energy")

ax.set_xlim([0, len(energy)//5])
ax.set_ylim([-0.1, 1.25 * (max_e - min_e)])
ax.set_yticklabels([])

plt.tight_layout()
plt.savefig('figures/Figure5/energy.eps')
plt.show()

### Set up Plotting of Projections

In [ ]:
def plot_embedding(
    embedding,
    conf_embedding,
    idx=range(len(traj)),
    xlabel=r'$PC_1$',
    ylabel=r'$PC_2$',
    coloring=True,
    cbar=True,
    savename=None,
):
    """
    Helper function to make regression plots in later sections
    
    Parameters
    ---------
    embedding: projection of the simulation data, size (N, n_components)
    conf_embedding: projection of the conformers, size (5, n_components)
    idx: subselection of the simulation frames, array of int
    xlabel: label for the x-axis, string
    ylabel: label for the y-axis, string
    coloring: whether or not to color datapoints by energies, boolean
    cbar: whether or not to include a colorbar, boolean
    
    """
    if not cbar:
        fig, ax = plt.subplots(1, figsize=(4, 4))
    else:
        fig, (ax, cax) = plt.subplots(1,2, figsize=(10.5, 8), gridspec_kw=dict(width_ratios=(6,1)))

    p = ax.scatter(*embedding[:, :2].T,
                   zorder=-1,
                   c=energy[idx]-min_e if coloring else 'grey',
                   vmax=max_e-min_e,
                   vmin=0,
                   cmap=cmap,
                   s=4)
    ax.scatter(
        *conf_embedding[:, :2].T,
        c=c_energy-min_e if coloring else 'grey',
        s=200,
        ec='k',
        lw=2,
        vmax=max_e-min_e,
        vmin=0,
        cmap=cmap,
    )
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)

    if cbar and coloring:
        plt.colorbar(p, cax=cax, label='Energy, eV')
        fig.subplots_adjust(hspace=1)
    if savename is not None:
        ax.set_xticklabels([])
        ax.set_yticklabels([])
        plt.savefig(savename)

### Create SOAP descriptors 
We create soap descriptor with the help of `rascal` library as below. The main important parameters are 

- `interaction_cutoff=2.5`: we are considering 2.5A radius of sphere for describing each atom environments. For cyclohexane, this includes both the nearest and second nearest carbons from each carbon center
- `max_radial=6`: expand over 6 radial GTO bases
- `max_angular=9`: expand over the first 9 spherical harmonics
- `gaussian_sigma_constant=0.3`: assume each atom has a gaussian of size sigma=0.3 imposed on its lattice site

Because we are focused on structure-level analysis, we average the descriptors for each configuration across all environmental centers and normalize to remove the feature means (we do the normalization explicitly in order to normalize both set of vectors by the same factors).

For more information on SOAP vectors and their implementation in librascal, we point the readers to [(Bartòk 2012)](https://journals.aps.org/prb/pdf/10.1103/PhysRevB.87.184115), [(Musil 2017)](https://aip.scitation.org/doi/full/10.1063/1.5090481), [(Musil 2021)](https://aip.scitation.org/doi/full/10.1063/5.0044689), and [(Goscinski 2021)](https://aip.scitation.org/doi/full/10.1063/5.0057229).

In [ ]:
hypers = {
    "interaction_cutoff": 2.5,
    "max_radial": 6,
    "max_angular": 9,
    "gaussian_sigma_constant": 0.3,
    "gaussian_sigma_type": "Constant",
    "cutoff_smooth_width": 0.8,
    "radial_basis": "GTO",
    "global_species": [1, 6],
    "expansion_by_species_method": "user defined",
    "normalize": False
}

soap = SOAP(**hypers)
normalizer = StandardFlexibleScaler(column_wise=False)

soaps = normalizer.fit_transform(soap.transform(traj).get_features(soap))
split_soaps = np.split(soaps, len(traj))
mean_soaps = np.mean(split_soaps, axis=1)

conf_split_soaps = np.array([split_soaps[ci] for ci in conf_idx])
conf_mean_soaps = np.mean(conf_split_soaps, axis=1)

# saving soap vectors
np.savez('./cyclohexane_data/soap_vectors.npz',
         mean_soaps=mean_soaps,
         soaps=soaps,
         conf_split_soaps=conf_split_soaps,
         conf_mean_soaps=conf_mean_soaps)

print(soaps.shape)

**Using these SOAP vectors, we can detect the phase transition even when we do not know the energetics, solely the configurations.**

## Analysis at via Linear Methods

### Linear Principal Components Analysis

In [ ]:
from sklearn.decomposition import PCA

pca = PCA()
pca.fit(mean_soaps)

t_pca = pca.transform(mean_soaps)
t_pca_conf = pca.transform(mean_soaps[conf_idx])

plot_embedding(t_pca, t_pca_conf, savename='figures/Figure5/pca.eps')

Even when our PCA is not easily interpretable, we can use it towards data compression by looking at the variance contained in the components:

In [ ]:
plt.loglog(pca.explained_variance_ratio_)
plt.gca().set_xlabel(r'$n_{PC}$')
plt.gca().set_ylabel("Explained Variance Ratio")

n_pca = np.where(np.cumsum(pca.explained_variance_ratio_) > 0.999)[0][0]
plt.axvline(n_pca, c='k', linestyle='--')
print(
    "This shows that we can retain most of the variance (>99.9%) in just {} vectors. We'll use this as our descriptor in some algorithms below for complexity's sake."
    .format(n_pca))

In [ ]:
pca_desc = t_pca[:, :n_pca]
conf_pca_desc = t_pca_conf[:, :n_pca]

In [ ]:
np.savez('cyclohexane_data/pca.npz', pca=pca_desc, pca_conf=conf_pca_desc)

### t-SNE

PCA is not intended as a clustering algorithm -- it just sometimes work out to give nice clusters.
Let's employ one of the most popular non-linear dimensionality reduction algorithm in ML field `T-distributed Stochastic Neighbor Embedding (t-SNE)` to obtain 2 dimensional representation of our descriptor space. 

Here we can see how increasing the perplexity (number of expected neighbors) changes the layout of the projection.

In [ ]:
perplexities = np.logspace(0, 2, 6, dtype=int)
fig, ax = plt.subplots(1,
                       len(perplexities),
                       figsize=(4 * len(perplexities), 4),
                      )

for i, perp in enumerate(tqdm(perplexities)):
    tsne = TSNE(
        n_components=2,  # number of components to project across
        perplexity=
        perp,  # amount of neighbors one point is posited to have... play around with this!
        metric="euclidean",  # distance metric
        n_jobs=2,  # parallelization
        random_state=42,
        verbose=False,
    )
    t_tsne = tsne.fit(pca_desc)
    ax[i].scatter(*t_tsne.T, c=energy, cmap=cmap, s=2)
    ax[i].axis('off')
    ax[i].set_title("Perplexity = {}".format(perp))
plt.show()

In [ ]:
tsne = TSNE(
    n_components=2,  # number of components to project across
    perplexity=50,  # amount of neighbors one point is posited to have... play around with this!
    metric="euclidean",  # distance metric
    n_jobs=2,  # parallelization
    random_state=42,
    verbose=False,
)

In [ ]:
t_tsne = tsne.fit(pca_desc)
t_tsne_conf = t_tsne.transform(conf_pca_desc)
plot_embedding(t_tsne,
               t_tsne_conf,
               xlabel=r'$t-SNE_1$',
               ylabel=r'$t-SNE_2$',
               cbar=False,
               savename='figures/Figure5/tsne.eps')

In [ ]:
np.savez('cyclohexane_data/tsne.npz', tsne=t_tsne, tsne_conf=t_tsne_conf)

### UMAP

In [ ]:
import umap

UMAP _should_ obtain similar results to t-sne, but with a shorter compute time. However, you will note a greater stochasticity to the projection when using a smaller number of neighbors -- this is due to the disconnection of the locally constructed manifolds.

In [ ]:
nneigh = np.maximum(2, np.logspace(0, 1.7, 5, dtype=int))
fig, ax = plt.subplots(1,
                       len(nneigh),
                       figsize=(4*len(nneigh), 4),
                      )

for i, n in enumerate(tqdm(nneigh)):
    um = umap.UMAP(n_components=2, n_neighbors=n, init='random')
    um.fit(pca_desc)
    t_um = um.transform(pca_desc)
    ax[i].scatter(*t_um.T, c=energy, cmap=cmap, s=2)
    ax[i].axis('off')
    ax[i].set_title("# Neighbors = {}".format(n))
plt.show()

In [ ]:
um = umap.UMAP(n_components=2, n_neighbors=50)
um.fit(pca_desc)

In [ ]:
t_um = um.transform(pca_desc)
t_um_conf = um.transform(conf_pca_desc)
plot_embedding(t_um,
               t_um_conf,
               xlabel='UMAP(1)',
               ylabel='UMAP(2)',
               cbar=False,
               savename='figures/Figure5/umap.eps')

In [ ]:
np.savez('cyclohexane_data/umap.npz', umap=t_um, umap_conf=t_um_conf)

### PCovR

PCovR is a dimensionality reduction technique that balances supervised and unsupervised learning. 
Here, we can use the parameter `mixing` to weight between unsupervised (0) and supervised (1). For more information, see the set of [PCovR / KPCovR tutorials](https://github.com/cosmo-epfl/kernel-tutorials), [(de Jong, 1992)](https://www.sciencedirect.com/science/article/pii/016974399280100I), and [(Helfrecht, 2020)](https://iopscience.iop.org/article/10.1088/2632-2153/aba9ef).

In [ ]:
from skcosmo.decomposition import PCovR
from sklearn.linear_model import RidgeCV

# In PCovR, we first need to prepare a regression model
# This prevents our decomposition from overfitting on the targets
y = StandardFlexibleScaler(column_wise=True).fit_transform(np.vstack(energy))
Yp = RidgeCV(cv=2, alphas=np.logspace(-10, 2),
             fit_intercept=False).fit(mean_soaps, y).predict(mean_soaps)

In [ ]:
pcovr = PCovR(n_components=2, mixing=0.5)
t_pcovr = pcovr.fit(pca_desc, Yp).transform(pca_desc)
conf_t_pcovr = pcovr.transform(conf_pca_desc)

In [ ]:
plot_embedding(t_pcovr,
               conf_t_pcovr,
               cbar=False,
               xlabel=r'$PCov_1$',
               ylabel=r'$PCov_2$',
               savename='figures/Figure5/pcovr.eps')

In [ ]:
np.savez('cyclohexane_data/pcovr.npz', pcovr=t_pcovr, pcovr_conf=conf_t_pcovr)

In [ ]:
fig, axes = plt.subplots(1, 5, figsize=(15, 3), sharex=True, sharey=True)
for ax, mixing in zip(axes, [0.0, 0.1, 0.5, 0.9, 1.0]):
    pcovr = PCovR(n_components=2, mixing=mixing)
    T = pcovr.fit(mean_soaps, Yp).transform(mean_soaps)

    ax.scatter(T[:, 0],
               T[:, 1],
               c=energy,
               zorder=-1,
               vmax=max_e,
               vmin=min_e,
               cmap=cmap,
               s=4)

    ax.set_title(f'mixing={round(mixing, 4)}')
    ax.set_xlabel(r"$PCov_1$")
    
axes[0].set_ylabel(r"$PCov_2$")
axes[0].set_xticklabels([])
axes[0].set_yticklabels([])

plt.show()

## Non-Linear Dimensionality Reduction
### Kernel Principal Components Analysis

So far we've seen similar results from most linear methods -- a plot which shows the high-energy conformers at the ends of the projection and some middling of low-energy conformers in the center. While this demonstrates the span of structures in the simulation, our low-energy conformers are often compressed in the middle of the mapping, as they are far more similar than the high-energy oscillations. Here, we can expand the enumeration of the low-energy conformers by taking a non-linear distance of the structures.

In [ ]:
from sklearn.decomposition import KernelPCA
from skcosmo.preprocessing import KernelNormalizer

When building a kernel on **averaged** representations, we construct our kernel such that

$$K(\mathcal{X}_A, \mathcal{X}_{B}) = \frac{\sum_{i \in A} \sum_{j \in B} k\left(\mathcal{X}_i, \mathcal{X}_j\right)}{N_A N_B}$$

where the kernel between structures $A$ and $B$ is the average of the kernels of each of their atomic environments. With non-linear kernels, this is not equal to the kernel of the averaged representations, i.e.

$$K(\mathcal{X}_A, \mathcal{X}_B) \neq k\left(\sum_{i\in A}\frac{\mathcal{X}_i}{N_A}, \sum_{j\in B}\frac{\mathcal{X}_j}{N_B}\right)$$


**If you want to speed up this section, which can take a while:** just un-comment the lines marked "simplify" and comment those marked "full calculation". 

In [ ]:
idx = np.random.choice(range(len(traj)), 10000, replace=False) # simplify
atom_idx = np.concatenate([range(i*6, (i+1)*6) for i in idx]) # simplify

# idx = np.arange(len(traj)) # full calculation
# atom_idx = np.arange(len(soaps)) # full calculation

In [ ]:
gamma = 5.0
m = 6

kernel = partial(pairwise_kernels, metric='rbf', gamma=gamma, n_jobs=6)

# because we know that atom-ordering is conserved in this trajectory,
# we'll only consider comparisons between identical atom indices
# this would correspond with a `best-match` kernel
K_raw = np.zeros((len(idx), len(idx)))

my_soaps = np.reshape([split_soaps[i] for i in idx],
                      (len(idx) * m, -1))

for i in tqdm(range(len(idx))):
    Ki = kernel(my_soaps[i * m:(i + 1) * m], my_soaps[i * m:])
    for j in range(i, len(idx)):
        K_raw[i, j] = K_raw[j, i] = np.mean(
            np.diag(Ki[:, (j - i) * m:(j - i + 1) * m]))

In [ ]:
# we also construct a kernel between our simulation data and conformers
# in order to embed them in the same latent space
conf_K_raw = np.zeros((5, len(idx)))

for i, ci in tqdm(enumerate(conf_idx)):
    ki = kernel(split_soaps[ci], my_soaps)
    for j in range(len(idx)):
        conf_K_raw[i, j] = np.mean(np.diag(ki[:, j * m:(j + 1) * m]))

Working with centered kernels is crucial (more on this in [Helfrecht, 2020](https://iopscience.iop.org/article/10.1088/2632-2153/aba9ef)).

In [ ]:
kn = KernelNormalizer(with_trace=False).fit(K_raw)
K = kn.transform(K_raw)
conf_K = kn.transform(conf_K_raw)

In [ ]:
kpca = KernelPCA(n_components=n_pca, kernel='precomputed')
kpca.fit(K)

t_kpca = kpca.transform(K)
t_kpca_conf = kpca.transform(conf_K)

plot_embedding(t_kpca,
               t_kpca_conf,
               idx=idx,
               xlabel=r'$KPCA_1$',
               ylabel=r'$KPCA_2$',
               cbar=False,
               savename='figures/Figure5/kpca.eps')

Here we can see an 'unfolding' of the trajectory, where the high energy states are put closer together.

In [ ]:
# properties = {"KPCA": t_kpca,
#              }
# widget = chemiscope.show([traj[i] for i in idx], properties)
# widget

### Kernel PCovR
PCovR can also be extended for non-linear distance metrics, here we can use our kernel computed in 2.2.

In [ ]:
from skcosmo.decomposition import KernelPCovR
from sklearn.kernel_ridge import KernelRidge

y = StandardFlexibleScaler(column_wise=True).fit_transform(np.vstack(energy))
Yp = KernelRidge(kernel='precomputed', alpha=1E-4).fit(K, y[idx]).predict(K)
kpcovr = KernelPCovR(n_components=2, mixing=0.5)
t_kpcovr = kpcovr.fit(K, Yp).transform(K)
conf_t_kpcovr = kpcovr.transform(conf_K)

In [ ]:
plot_embedding(t_kpcovr, conf_t_kpcovr, idx=idx,
               cbar=False,
               xlabel=r'$KPCov_1$',
               ylabel=r'$KPCov_2$',
               savename='figures/Figure5/kpcovr.eps')

In [ ]:
fig, axes = plt.subplots(1, 5, figsize=(15, 3), sharex=True, sharey=True)
for ax, mixing in zip(axes, [0.0, 0.1, 0.5, 0.9, 1.0]):
    kpcovr = KernelPCovR(n_components=2, mixing=mixing)
    T = kpcovr.fit(K, Yp).transform(K)

    ax.scatter(T[:, 0],
               T[:, 1],
               c=energy[idx],
               zorder=-1,
               vmax=max_e,
               vmin=min_e,
               cmap=cmap,
               s=4)

    ax.set_title(f'mixing={round(mixing, 4)}')
    ax.set_xlabel("KPCov_1")
    
axes[0].set_ylabel("KPCov_2")
axes[0].set_xticklabels([])
axes[0].set_yticklabels([])

plt.show()

## Further Exploration
Now we can export a chemiscope to play around with all of this on [chemiscope.org](chemiscope.org)!

In [ ]:
import chemiscope
properties = {"PCA": t_pca[:, :5],
              "t-SNE": t_tsne,
              "UMAP": t_um,
              "PCovR": t_pcovr,
             }
widget = chemiscope.show(traj, properties)
widget.save('cyclohexanes.json')

In [ ]:
import chemiscope
properties = {"PCA": t_pca[:, :5],
              "t-SNE": t_tsne,
              "UMAP": t_um,
              "PCovR": t_pcovr,
             }
widget = chemiscope.show(traj, properties)
widget.save('cyclohexanes.json')